1) Carrega bibliotecas e DataFrame; configura visualização

In [9]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import plotly.express as px
import requests

# Configura visualização
pd.set_option('display.max_columns', None)
plt.style.use('seaborn-v0_8')
sns.set_theme()

caminho_dados = 'dados/acidentes.csv'

try:
    df = pd.read_csv(caminho_dados, sep=';', encoding='utf-8')
    print("CSV original lido com sucesso!")
except FileNotFoundError:
    print(f"Erro: Arquivo '{caminho_dados}' não encontrado. Verifique o caminho.")
except Exception as e:
    print(f"Ocorreu um erro: {e}")

CSV original lido com sucesso!


2) Entende DataFrame

In [10]:
# Lista as cinco primeira linhas
print("\nCinco primeiras linhas:")
print(df.head())

# Mostra o número de linhas e colunas
print("\nDimensões do Dataset (linhas, colunas)")
print(df.shape)

# Mostra tipo de dados, valores não nulos
print("\nInformações gerais do DataFrame:")
df.info()

# Conta valores ausentes por coluna
print("\nContagem de Valores Ausentes por Coluna")
print(df.isnull().sum())



Cinco primeiras linhas:
         data_extracao  predial1  queda_arr                 data  feridos  \
0  2025-06-01 01:33:13       0.0        0.0  2020-10-17 00:00:00        1   
1  2025-06-01 01:33:13     598.0        0.0  2020-01-01 00:00:00        1   
2  2025-06-01 01:33:13    1271.0        0.0  2020-01-01 00:00:00        1   
3  2025-06-01 01:33:13    1901.0        0.0  2020-01-02 00:00:00        2   
4  2025-06-01 01:33:13    3302.0        0.0  2020-01-02 00:00:00        1   

   feridos_gr  mortes  morte_post  fatais  auto  taxi  lotacao  onibus_urb  \
0           0       0           0       0     3     0        0           0   
1           0       0           0       0     0     1        0           0   
2           1       0           0       0     1     0        0           0   
3           0       0           0       0     0     0        0           0   
4           0       0           0       0     1     0        0           0   

   onibus_met  onibus_int  caminhao  moto  

3) Limpa nomes colunas; remove 'data_extracao', 'consorcio', 'latitude' e 'longitude'

In [11]:
# Remove espaços em branco no início/fim dos nomes das colunas (boa prática)
df.columns = df.columns.str.strip()

# Remove colunas não utilizadas
df = df.drop(columns=['data_extracao', 'consorcio', 'latitude', 'longitude'], errors='ignore')

# Lista nome de colunas
print(df.columns)


Index(['predial1', 'queda_arr', 'data', 'feridos', 'feridos_gr', 'mortes',
       'morte_post', 'fatais', 'auto', 'taxi', 'lotacao', 'onibus_urb',
       'onibus_met', 'onibus_int', 'caminhao', 'moto', 'carroca', 'bicicleta',
       'outro', 'cont_vit', 'ups', 'patinete', 'idacidente', 'log1', 'log2',
       'tipo_acid', 'dia_sem', 'hora', 'noite_dia', 'regiao'],
      dtype='object')


4) Transforma tipos; remove inválidos

In [12]:
# Converte 'data' para datetime
df['data'] = pd.to_datetime(df['data'], errors='coerce')

# Remove inválidos 'data', 'hora', 'log1', 'regiao'
df = df.dropna(subset=['data', 'hora', 'log1', 'regiao'])

# Remove data fora do escopo (2020-2025)
df = df[(df['data'] >= '2020-01-01') & (df['data'] <= '2025-04-01')]
print("Datas fora do escopo (2020-2025) removidas com sucesso.")

# Remove chaves duplicadas
df = df.drop_duplicates(subset='idacidente')

# Transforma colunas tipos de colunas em Categoria
col_cat = ['regiao', 'tipo_acid', 'dia_sem', 'noite_dia']
df[col_cat] = df[col_cat].astype('category')

# Transforma colunas tipos de colunas Inteiro
col_int = [c for c in [
  'queda_arr', 'feridos', 'feridos_gr', 'mortes',
  'morte_post', 'fatais', 'auto', 'taxi', 'lotacao', 'onibus_urb',
  'onibus_met', 'onibus_int', 'caminhao', 'moto', 'carroca', 'bicicleta',
  'outro', 'cont_vit', 'ups', 'patinete', 'idacidente', 'predial1'
  ] if c in df.columns]
for c in col_int:
    df[c] = pd.to_numeric(df[c], errors='coerce').astype('Int32')

# Transforma colunas tipos de colunas em Float
col_float = [c for c in ['longitude', 'latitude'] if c in df.columns]
for c in col_float:
    df[c] = pd.to_numeric(df[c], errors='coerce').astype('float32')

# Transforma colunas tipos de colunas em String
col_str = [c for c in ['log1', 'log2'] if c in df.columns]
for c in col_str:
    df[c] = df[c].astype('string')

# Transforma coluna 'hora' para datetime
df['hora'] = pd.to_datetime(df['hora'], errors='coerce')

     
print("\nInformações após:")
df.info()
df.describe(include='all')
df.head()

Datas fora do escopo (2020-2025) removidas com sucesso.

Informações após:
<class 'pandas.core.frame.DataFrame'>
Index: 68837 entries, 0 to 69520
Data columns (total 30 columns):
 #   Column      Non-Null Count  Dtype         
---  ------      --------------  -----         
 0   predial1    64799 non-null  Int32         
 1   queda_arr   68837 non-null  Int32         
 2   data        68837 non-null  datetime64[ns]
 3   feridos     68837 non-null  Int32         
 4   feridos_gr  68837 non-null  Int32         
 5   mortes      68837 non-null  Int32         
 6   morte_post  68837 non-null  Int32         
 7   fatais      68837 non-null  Int32         
 8   auto        68837 non-null  Int32         
 9   taxi        68837 non-null  Int32         
 10  lotacao     68837 non-null  Int32         
 11  onibus_urb  68837 non-null  Int32         
 12  onibus_met  68837 non-null  Int32         
 13  onibus_int  68837 non-null  Int32         
 14  caminhao    68837 non-null  Int32         
 15  

/tmp/ipykernel_1731/3886176034.py:39: UserWarning: Could not infer format, so each element will be parsed individually, falling back to `dateutil`. To ensure parsing is consistent and as-expected, please specify a format.
  df['hora'] = pd.to_datetime(df['hora'], errors='coerce')


,predial1,queda_arr,data,feridos,feridos_gr,mortes,morte_post,fatais,auto,taxi,lotacao,onibus_urb,onibus_met,onibus_int,caminhao,moto,carroca,bicicleta,outro,cont_vit,ups,patinete,idacidente,log1,log2,tipo_acid,dia_sem,hora,noite_dia,regiao
0,0,0,2020-10-17,1,0,0,0,0,3,0,0,0,0,0,0,1,0,0,0,1,5,0,190816,R MARCOS MOREIRA,R GASTON ENGLERT,ABALROAMENTO,SÁBADO,2025-08-21 19:00:00,NOITE,NORTE
1,598,0,2020-01-01,1,0,0,0,0,0,1,0,0,0,0,0,1,0,0,0,1,5,0,669089,AV BENTO GONCALVES,<NA>,ABALROAMENTO,QUARTA-FEIRA,2025-08-21 03:00:00,NOITE,LESTE
2,1271,0,2020-01-01,1,1,0,0,0,1,0,0,0,0,0,0,0,0,0,0,1,5,0,669097,AV INDEPENDENCIA,<NA>,ATROPELAMENTO,QUARTA-FEIRA,2025-08-21 23:00:00,NOITE,LESTE
3,1901,0,2020-01-02,2,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,1,5,0,669098,AV EDUARDO PRADO,<NA>,ATROPELAMENTO,QUINTA-FEIRA,2025-08-21 00:05:00,NOITE,SUL
4,3302,0,2020-01-02,1,0,0,0,0,1,0,0,0,0,0,0,1,0,0,0,1,5,0,669099,AV TERESOPOLIS,<NA>,ABALROAMENTO,QUINTA-FEIRA,2025-08-21 09:00:00,DIA,SUL


5) Cria novas variáveis e DataFrames necessários

In [15]:
# Cria coluna 'hora_int'
df['hora_int'] = pd.to_datetime(df['hora'], format='%H:%M:%S', errors='coerce').dt.hour

# Cria coluna 'data_hora'
df['data_hora'] = pd.to_datetime(df['data'].astype(str) \
    + ' ' + df['hora'].dt.strftime('%H:%M:%S'), errors='coerce')

print(df.head())
df.info()





   predial1  queda_arr       data  feridos  feridos_gr  mortes  morte_post  \
0         0          0 2020-10-17        1           0       0           0   
1       598          0 2020-01-01        1           0       0           0   
2      1271          0 2020-01-01        1           1       0           0   
3      1901          0 2020-01-02        2           0       0           0   
4      3302          0 2020-01-02        1           0       0           0   

   fatais  auto  taxi  lotacao  onibus_urb  onibus_met  onibus_int  caminhao  \
0       0     3     0        0           0           0           0         0   
1       0     0     1        0           0           0           0         0   
2       0     1     0        0           0           0           0         0   
3       0     0     0        0           0           0           0         0   
4       0     1     0        0           0           0           0         0   

   moto  carroca  bicicleta  outro  cont_vit  ups 

6) Puxa dados de precipitação baseado nas regiões

In [ ]:
# Analisa regiões
df['regiao'].unique()

# Cria dicionário coordenadas por região
coord = {
    "Norte": (-29.987, -51.165),
    "Leste": (-30.040, -51.160),
    "Centro": (-30.027, -51.220),
    "Sul": (-30.120, -51.230)
}

['NORTE', 'LESTE', 'SUL', 'CENTRO']
Categories (4, object): ['CENTRO', 'LESTE', 'NORTE', 'SUL']